In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!pip install pandas
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd huggan/pytorch/cyclegan
!pip install wandb
!wandb login
!accelerate config

In [ ]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os



hyperparams_dcgan = {
    
    'lr':[0.0002], 
    'latent_dim':[100],
        'generator_hidden_size':[64],
    'discriminator_hidden_size':[64],
    
    
        'image_size':[64], 
        'num_channels':[3], 
    'dataset':['Chris1/cryptopunks'],  
'num_workers':[8], 
     'batch_size':[8],
    
    'num_epochs':[200],  
        'beta1':[0.5], 
 #'fp16':[False],
    'mixed_precision':['no'], 
    'output_dir':['./output'],
    #'cpu':[False], 
    'wandb':[True],
    'logging_steps':[50],
    'push_to_hub':[True], 
    'model_name':['test'],
     'organization_name':['Chris1'], 
    
    
                     }


In [ ]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)



for idx, grid in enumerate(ParameterGrid(hyperparams_dcgan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments/{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
        call(call_params)

INFO:__main__:| epoch: 1.900e+01| discriminator_loss: 2.632e-03| generator_loss: 7.618e+00| D_x: 9.978e-01| D_G_z1: 9.919e-04| D_G_z2: 5.904e-04
